参考：
* https://pytorch.apachecn.org/docs/1.4/blitz/neural_networks_tutorial.html
* https://mp.weixin.qq.com/s/kDGHQQ_wRQXds34HHCQNOQ  神经网络
* 细节：https://zhuanlan.zhihu.com/p/315092879


可以使用`torch.nn`包来构建神经网络.

我们已经介绍了`autograd`包，**`nn`包则依赖于`autograd`包来定义模型并对它们求导。**
* **一个`nn.Module`包含各个层和一个`forward(input)`方法，该方法返回`output`。**

例如，下面这个神经网络可以对数字进行分类：

![convnet](https://pytorch.org/tutorials/_images/mnist.png)

这是一个简单的前馈神经网络(feed-forward network）。它接受一个输入，然后将它送入下一层，一层接一层的传递，最后给出输出。

### 一个神经网络的典型训练过程如下：

* 定义包含一些可学习参数(或者叫权重）的神经网络
* 在输入数据集上迭代
* 通过网络处理输入
* 计算loss(输出和正确答案的距离）
* 将梯度反向传播给网络的参数
* 更新网络的权重，一般使用一个简单的规则：`weight = weight - learning_rate * gradient`


## 定义网络

在Pytorch中创建一个神经网络需要继承torch.nn.module定义一个类，在这个类中主要是有两个函数需要重写，
* 一个是__init__()函数，需要使用super(Net, self).__init__()来方便后续调用父类的函数。__init__()中还需要定义好需要使用到的所有的神经网络的层。
* 另外一个函数是forward()函数，他是前向计算的函数，定义好前向计算就表示从输入到输出的计算图定义好了，反向传播根据前向计算图自动计算的，不用自己定义，大大方便了网络的创建。以下代码就是创建了一个有两个卷积层，三个全连接的神经网络结构。


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):  # 定义一些函数
        super(Net, self).__init__()
        # 输入图像channel：1；输出channel：6；5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)  # 卷积   nn.Conv2d 接受一个4维的张量，即nSamples(BatchSize) x nChannels x Height x Width
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b  仿射操作
        self.fc1 = nn.Linear(16 * 5 * 5, 120)   # 输出channel：16；5x5卷积核   全连接
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 2x2 Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果是方阵,则可以只使用一个数字进行定义
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) # 修改维度，调用num_flat_features函数
        x = F.relu(self.fc1(x))  # relu
        x = F.relu(self.fc2(x))  # relu
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):  # flat 平面  被上面forward调用   这里的num_flat_features()函数可以认为是一个helper函数。
        size = x.size()[1:]  # 除去批处理维度的其他所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net) # 打印这个网络信息

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


我们只需要定义 forward 函数，backward函数会在使用autograd时自动定义，backward函数用来计算导数。我们可以在 forward 函数中使用任何针对张量的操作和计算。





In [20]:
# 一个模型的可学习参数可以通过net.parameters()返回
params = list(net.parameters())
print(len(params)) # 10
print(params[0].size())  # conv1's .weight   torch.Size([6, 1, 5, 5])

10
torch.Size([6, 1, 5, 5])


In [21]:
# 让我们尝试一个随机的32x32的输入。注意:这个网络(LeNet）的期待输入是32x32的张量。如果使用MNIST数据集来训练这个网络，要把图片大小重新调整到32x32。

input = torch.randn(1, 1, 32, 32)  # 声明一个1x1x32x32的4维张量作为网络的输入 ,nSamples(BatchSize) x nChannels x Height x 
out = net(input)  # 输入四维
print(out)



tensor([[-0.0156, -0.0351,  0.0496,  0.0336,  0.0285,  0.0076,  0.1551, -0.0653,
          0.1051, -0.0059]], grad_fn=<AddmmBackward>)


目前为止，我们讨论了：

* 定义一个神经网络
* 处理输入调用backward

还剩下：

* 计算损失
* 更新网络权重

## 损失函数

一个损失函数接受一对(output, target)作为输入，计算一个值来估计网络的输出和目标值相差多少。

nn包中有很多不同的损失函数。nn.MSELoss是比较简单的一种，它计算输出和目标的均方误差(mean-squared error）。

```
criterion = nn.MSELoss()
loss = criterion(output, target)

```



In [22]:
output = net(input)
target = torch.randn(10)  # 本例子中使用模拟数据
target = target.view(1, -1)  # 使目标值与数据值尺寸一致
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.4827, grad_fn=<MseLossBackward>)


In [23]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## 反向传播


In [24]:
net.zero_grad()     # 清零所有参数(parameter）的梯度缓存

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()  # loss的反向传播

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0102,  0.0089,  0.0010, -0.0100, -0.0012,  0.0031])


## 更新权重

In [25]:
import torch.optim as optim

# 创建优化器(optimizer）
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 在训练的迭代中：
optimizer.zero_grad()   # 清零梯度缓存
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 更新参数